# Licence Plate Blurring

Working with pretrained model, License Plate Detector

In [1]:
import os
from ultralytics import YOLO 
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

c:\Users\santi\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_best_path = "directory to model"
model = YOLO(model_best_path)

In [22]:
image_name = "image_name.jpg"
image_path = "directory of image"
image_path_complete = os.path.join(image_path,image_name)

def image_licence_blur(image_path_complete):
    result = model.predict(image_path_complete,visualize=True ,conf=0.25,augment=True)
    for r in result:
        boxes = r.boxes.xyxy.cpu().numpy()
        r.show()
    print(boxes)

    image = cv2.imread(image_path_complete)

    if boxes is not None:
        for b in boxes:
            x1,y1,x2,y2 = b[0],b[1],b[2],b[3]
            x1 = int(x1)
            y1 = int(y1)
            x2 = int(x2)
            y2 = int(y2)
            image = cv2.rectangle (image,(x1,y1),(x2,y2), 0)
            image[y1:y2,x1:x2, :]= cv2.blur(image[y1:y2,x1:x2, :],(50,50),cv2.BORDER_REFLECT)
    plt.imshow(image)

    output_path = "output path"
    cv2.imwrite(os.path.join(output_path, image_name),image)
    return "Image Processed"

In [26]:
video_name = "VIDEO_NAME.mp4"
video_path = "video path"
video_path_complete = os.path.join(video_path,video_name)
output_path = "output path"

def video_licence_blur(video_path_complete):
    cap = cv2.VideoCapture(video_path_complete)
    scores = []

    # Information of output video
    fps = 25  # Fotogramas por segundo
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')  # Codec de video
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) # Ancho del frame
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) # Alto del frame
    out = cv2.VideoWriter(os.path.join(output_path, "output_name.mp4"), fourcc, fps, (width, height)) # Output Name

    while cap.isOpened():
        # Read a frame from the video
        success, frame = cap.read()
        # If any video is being reading
        if success:
            # Run model
            result = model.predict(frame,visualize=True ,conf=0.40,augment=True)
            for r in result:
                boxes = r.boxes.xyxy.cpu().numpy()

            if boxes is not None:
                for box in boxes:
                    x1, y1, x2, y2 = map(int, box)
                    frame = cv2.rectangle (frame,(x1,y1),(x2,y2), 0)
                    frame[y1:y2,x1:x2, :]= cv2.blur(frame[y1:y2,x1:x2, :],(50,50),cv2.BORDER_REFLECT)
            out.write(frame)

        else:
            break

    # Release resources
    cap.release()
    out.release()
    return "Video Processed"